# Automated Travel Poster Generation System

## Project Overview

This project aims to develop an automated system for generating travel posters using Generative AI technologies. The intended users are travel agencies and content creators who need to produce engaging and visually appealing marketing materials efficiently. This system leverages text and image generation models to automate the creation of customized travel posters, streamlining content production and enhancing creative outputs.

## Step 1: Business Use Case Identification

### Objective

The primary objective is to create a tool that enables rapid production of travel posters, which can be used to enhance online engagement and marketing efforts. By automating this process, we aim to reduce the time and cost associated with graphic design while allowing for high scalability and customization.

### Justification

In the digital age, visual content is key to capturing audience attention. Travel agencies and social media content creators often require quick turnaround and high volumes of diverse and eye-catching promotional materials. By automating poster creation, we provide a value-added solution that meets these market demands efficiently.

## Step 2: Model Selection and Setup

### Text Generation Model: GPT-2

**Choice Justification:**
- **Performance vs. Resource Efficiency:** GPT-2 offers a good balance, making it ideal for local deployment where resources like memory and processing power may be limited.
- **Ease of Use:** The `transformers` library provides an accessible interface to leverage this model without deep technical knowledge of its inner workings.

### Image Generation Model: Stable Diffusion

**Choice Justification:**
- **High-Quality Outputs:** Known for generating detailed and visually appealing images that can be customized through textual prompts.
- **Local Performance Optimization:** Compatible with MPS (Metal Performance Shaders), enabling efficient GPU acceleration on macOS devices.

### Implementation Details

In [15]:
!pip install diffusers --verbose

Using pip 23.3.1 from /Users/fulin/anaconda3/lib/python3.11/site-packages/pip (python 3.11)


In [ ]:
from transformers import pipeline
from diffusers import StableDiffusionPipeline
import torch

# I got a macbook so use mps firstly
device = "mps" if torch.backends.mps.is_available() else "cpu"

## System Integration and Workflow

### Generating Textual Content

First, the system generates a text description using GPT-2, tailored to the theme of "travel". This description guides the visual content creation.

In [ ]:
# text generator
text_generator = pipeline("text-generation", model="gpt2")

# generate a text
topic = "travel"
prompt = f"Creat a {topic} poster:"
text_output = text_generator(prompt, max_length=100)
generated_text = text_output[0]['generated_text']
print("Generated Text:", generated_text)

### Generating the Poster Image

Using the text description as a prompt, the Stable Diffusion model generates a corresponding image. This process is tuned to operate efficiently on local machines while maintaining high output quality.

In [ ]:
# image generator
image_generator = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", 
    torch_dtype=torch.float16
).to(device)

# generate the photo
image_output = image_generator(generated_text, num_inference_steps=50).images[0]

# show
image_output.show()

## Step 3: Model Optimization and Adaptation

### Objective

To improve the efficiency and performance of the generative models used (GPT-2 for text generation and Stable Diffusion for image generation) when deployed locally. The adaptation aims to reduce resource consumption (like memory and computation time) without significantly sacrificing the output quality.

### Methodology

1. **Fine-Tuning the GPT-2 Model:**
   - **Data Acquisition:** Identify and acquire a travel-related dataset. Possible sources include travel blogs, descriptions from travel websites, or a dataset from repositories like Kaggle.
   - **Training Objective:** Fine-tune GPT-2 on the travel dataset to enhance its ability to generate relevant and context-specific text for travel posters.
   - **Tools and Libraries:** Use `transformers` and `datasets` from Hugging Face for training and data handling.

2. **Optimizing Stable Diffusion:**
   - **Model Pruning:** Implement pruning techniques to reduce the size of the model while maintaining the quality of the generated images. This involves systematically removing parameters that have little impact on the output.
   - **Quantization:** Apply quantization to decrease the model’s precision from float32 to float16 or even int8, which can reduce the model size and speed up inference time, with minimal impact on image quality.

### Experimentation

- **Setup Experimentation Environment:** Ensure that all experiments are reproducible with version control for both data and model configurations.
- **Performance Metrics:** Track performance improvements in terms of inference time and resource utilization. Use qualitative assessments (like user studies) to measure any changes in output quality.

### Implementation

In [ ]:
# Example of fine-tuning GPT-2 with the Hugging Face 'Trainer' API
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Load and prepare the dataset
dataset = load_dataset('path_to_travel_dataset')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Set training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned-travel",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    push_to_hub=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test']
)

# Start fine-tuning
trainer.train()